<a href="https://colab.research.google.com/github/JohnYCLam/Statistics/blob/main/Summary_of_Statistical_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Z Test, T Test, $\chi^2$ Test

Wald's Test, Likelihood Ratio Test, Testing implicit hypothesis

Jarque Bera Test, D'Agostino and Pearson Test

Kolmogorov-Smirnov test, Kolmogorov-Lilliefors test

Quantile-Quantile (QQ) plots

$\chi^2$ goodness-of-fit test

Confidence Interval Demonstration


In [ ]:
import numpy as np
import scipy.stats as stats

In [ ]:
#Construct a confidence interval of level 95% for the mean p of Bernoulli(p)
alpha = 0.05
q_alpha2 = stats.norm.ppf(1-alpha/2)

#Generate synthetic data
n = 100
X = np.random.binomial(1, 0.5, n)
X_avg = X.mean()

#Confidence interval 1: replace p(1-p) by 1/4
I = [X_avg - q_alpha2/(2*n**0.5), X_avg + q_alpha2/(2*n**0.5)]
print(I)

#Confidence interval 2: replace p by X_avg
I = [X_avg - q_alpha2 * (X_avg*(1 - X_avg)/n)**0.5, X_avg + q_alpha2 * (X_avg*(1 - X_avg)/n)**0.5]
print(I)

#Confidence interval 3: Calculate exact range of p
a = n + q_alpha2**2
b = - (2*n*X_avg + q_alpha2**2)
c = n*X_avg**2
I = [(-b - (b**2 - 4*a*c)**0.5)/(2*a), (-b + (b**2 - 4*a*c)**0.5)/(2*a)]
print(I)

[0.4220018007729973, 0.6179981992270027]
[0.4220802307169153, 0.6179197692830847]
[0.42316577765223967, 0.6153544824194809]


In [ ]:
#Meaning of Confidence interval
I = []
trials = 1000
n = 100
p = 0.5
count = 0

for _ in range(trials):
  X = np.random.binomial(1, p, n)
  X_avg = X.mean()
  lower = X_avg - q_alpha2/(2*n**0.5)
  upper = X_avg + q_alpha2/(2*n**0.5)
  I.append([lower, upper])
  if lower > p or upper < p:
    count += 1
print(f'Out of the {trials} constructed confidence intervals at 95% confidence level,\n{count} of them do not contain p ({p})')

Out of the 1000 constructed confidence intervals at 95% confidence level,
55 of them do not contain p (0.5)


Z Test: Underlying distribution is (asymptotically) normal AND the population variance is known. Then Z test is equivalent to testing whether the mean lies in the confidence interval.

In [ ]:
#Generate synthetic data
trials = 1000
n = 100
p = 0.6
X = np.random.binomial(n, p, trials)
X_avg = X.mean()

z_score = (X_avg - n*p)/(p*(1-p))**0.5
z_score

0.02857738033247633

In [ ]:
import scipy.stats as stats

alpha = 0.05
z_critical = stats.norm.ppf(1-alpha/2)
if z_score > z_critical:
  print("Reject H_0")
else:
  print("Failed to reject H_0")

Failed to reject H_0
